 Ярков Тимофей максимович, КБ-4 \
 N = 21

In [83]:
N = 21
R = 17 * N ** 2 + 5 * N + 3
print(R)

7605


Теперь найдем последовательность чисел, которая порождается генератором.\
Так как создаем последовательность для тестов NIST, то зададим длину более 128 бит

In [84]:
x_0 = 11579
dec_sequence = [x_0]
for i in range(30):
    x_next = (7 * R * dec_sequence[i] + R ** 2 - 11 * R) % 127
    dec_sequence.append(x_next)
print(dec_sequence)

[11579, 112, 60, 59, 37, 61, 81, 13, 41, 22, 112, 60, 59, 37, 61, 81, 13, 41, 22, 112, 60, 59, 37, 61, 81, 13, 41, 22, 112, 60, 59]


Так как для проверки случайности последовательности необходимо использовать тесты NIST, которые принимают на вход бинарные последовательности, то переведем и нашу последовательность в двоичный вид

In [85]:
binary_data = ''.join(bin(n)[2:] for n in dec_sequence)
print(f"Длина последовательности: {len(binary_data)}")
print(f"последовательность: {binary_data}")

Длина последовательности: 192
последовательность: 101101001110111110000111100111011100101111101101000111011010011011011100001111001110111001011111011010001110110100110110111000011110011101110010111110110100011101101001101101110000111100111011


Проведем блок NIST статистических тестов.
1. Частотный побитовый тест +
2. Частотный блочный тест +
3. Тест на последовательность одинаковых битов ?
4. Тест на самую длинную последовательность единиц в блоке +
5. Тест рангов бинарных матриц ?
6. Спектральный тест ?
7. Тест на совпадение неперекрывающихся шаблонов +
8. Тест на совпадение перекрывающихся шаблонов +
9. Универсальный статистический тест Маурера +
10. Тест на линейную сложность +
11. Тест на периодичность ?
12. Тест приблизительной энтропии +
13. Тест кумулятивных сумм +
14. Тест на произвольные отклонения ?

In [86]:
from scipy.special import erfc
import numpy as np

def monobit_test(bin_data):
    """
    Выполняет побитный частотный тест на двоичных данных.
    :param bin_data: Двоичная строка для тестирования
    :return: p-значение из теста
    """
    count = 0
    # Если символ равен 0, вычитаем 1, иначе прибавляем 1
    for char in bin_data:
        if char == '0':
            count -= 1
        else:
            count += 1
    # Вычисляем p-значение
    sobs = count / np.sqrt(len(bin_data))
    p_val = erfc(abs(sobs) / np.sqrt(2))
    return p_val

def block_frequency_test(bin_data, block_size = 128):
    """
    Выполняет блочный частотный тест на двоичных данных.
    :param bin_data: Двоичная строка для тестирования
    :param block_size: Размер блока
    :return: p-значение из теста
    """
    # Разбиваем данные на блоки
    num_blocks = len(bin_data) // block_size
    blocks = [bin_data[i * block_size:(i + 1) * block_size] for i in range(num_blocks)]
    
    # Вычисляем пропорцию нулей и единиц в каждом блоке
    proportions = [block.count('1') / block_size for block in blocks]
    
    # Вычисляем chi-square статистику
    chi_square = 4 * block_size * sum((prop - 0.5)**2 for prop in proportions)
    
    # Вычисляем p-значение
    p_val = erfc(chi_square / (2 * np.sqrt(2 * num_blocks)))
    return p_val

print(f"1. Тест на моноблоч частоту: p-значение = {monobit_test(binary_data)}")
print(f"2. Тест на частоту блока: p-значение = {block_frequency_test(binary_data)}")


1. Тест на моноблоч частоту: p-значение = 0.0024367348089890093
2. Тест на частоту блока: p-значение = 0.008275325921954001
